# LangServe

- LangServe는 LangChain에서 제공하는 **서비스 배포 프레임워크**임.

- LangChain 애플리케이션을 **REST API로 변환**하여 서비스화함.

- **Runnable 객체**를 API로 배포하고 **스트리밍 응답**을 지원함.

- **웹 기반 Playgroud**(API 테스트용)와 **클라이언트 SDK**를 제공함.


---

## 1. LangServe 설치하기

- LangServe 설치(pip 사용):

    ```bash
    pip install "langserve[all]"
    ```

- LangServe 설치(uv 사용):

    ```bash
    uv add "langserve[all]"   
    ```

- langchain-cli 설치:

    ```bash
    uv add langchain-cli
    ```

`(1) LangServe 설치 확인`

In [1]:
import langserve
langserve.__version__

'0.3.1'

`(2) 환경 변수 로드`

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---
## 2. 간단한 LangServe 서버 만들기  

`(1) 서버 파일 생성`
- app 폴더에 별도 파일을 생성하여 LangServe 서버를 구현함. 
- server.py라는 이름으로 생성함.

    ```python
    # app/server.py
    from fastapi import FastAPI
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv

    # 환경변수 로드
    load_dotenv()

    from langserve import add_routes

    # FastAPI 서버를 설정
    app = FastAPI(
        title="LangChain Server",
        version="1.0",
        description="Spin up a simple api server using Langchain's Runnable interfaces",
    )

    # 라우팅 설정
    add_routes(
        app,
        ChatOpenAI(model="gpt-4.1-mini"),
        path="/openai",   # OpenAI 모델에 대한 경로
    )

    if __name__ == "__main__":
        import uvicorn

        uvicorn.run(app, host="localhost", port=8000)
    ```    

`(2) 서버 실행하기`


```bash
uv run python app/server.py
```

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0001.png)

`(3) Playground에서 테스트`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0002.png)

`(4) LangSmith에서 트레이싱하기`

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/langserve_screenshot_0003.png)

---
## 3. LangServe 클라이언트 사용  

LangServe는 다음과 같은 API 엔드포인트를 자동으로 생성합니다:

- `/openai/invoke`: 동기 호출
- `/openai/stream`: 스트리밍 호출
- `/openai/batch`: 배치 처리
- `/openai/playground`: 웹 기반 UI 플레이그라운드

`(1) API 호출하기`

In [3]:
# - `/openai/invoke`: 동기 호출

import requests

url = "http://localhost:8000/openai/invoke"
headers = {
    "Content-Type": "application/json",
}

data = {
    "input": "파이썬이란 무엇인가요?"
}

response = requests.post(url, headers=headers, json=data)
print(response.status_code)
print(response.json())

200
{'output': {'content': '파이썬(Python)은 1991년에 귀도 반 로썸이 개발한 고급 프로그래밍 언어입니다. 파이썬은 가독성이 높고 문법이 간결하여 초보자부터 전문가까지 널리 사용되고 있습니다. 또한 다양한 분야에서 활용되며, 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립트 작성 등 다목적으로 사용됩니다.\n\n파이썬의 주요 특징은 다음과 같습니다:\n- **쉬운 문법:** 자연어와 비슷한 문법 구조로 배우기 쉽습니다.\n- **인터프리터 언어:** 코드를 작성한 후 바로 실행해볼 수 있습니다.\n- **풍부한 라이브러리:** 다양한 기능을 수행하는 표준 라이브러리와 외부 패키지가 많아 개발 시간을 단축할 수 있습니다.\n- **객체지향 프로그래밍:** 객체지향뿐만 아니라 절차적 프로그래밍, 함수형 프로그래밍도 지원합니다.\n- **크로스 플랫폼:** Windows, macOS, Linux 등 다양한 운영체제에서 실행할 수 있습니다.\n\n이러한 이유로 파이썬은 교육, 연구, 산업계 등 많은 분야에서 사랑받는 프로그래밍 언어입니다.', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 271, 'prompt_tokens': 17, 'total_tokens': 288, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcm

`(2) Python에서 클라이언트 SDK 사용하기`

In [4]:
from langserve.client import RemoteRunnable

# 원격 실행 가능 객체를 생성
chain = RemoteRunnable("http://localhost:8000/openai")

# API 호출
response = chain.invoke("파이썬이란 무엇인가요?")
print(response)

content='파이썬(Python)은 읽기 쉽고 배우기 쉬운 고급 프로그래밍 언어 중 하나입니다. 1991년에 귀도 반 로섬(Guido van Rossum)이 개발했으며, 간결한 문법과 강력한 표준 라이브러리를 특징으로 합니다.\n\n주요 특징은 다음과 같습니다:\n- **가독성:** 코드가 간결하고 명확하여 초보자도 쉽게 이해할 수 있습니다.\n- **다양한 용도:** 웹 개발, 데이터 분석, 인공지능, 자동화, 과학 계산 등 다양한 분야에서 사용됩니다.\n- **풍부한 라이브러리:** 다양한 기능을 제공하는 표준 라이브러리와 서드파티 라이브러리가 많습니다.\n- **인터프리터 언어:** 작성한 코드를 한 줄씩 실행하며, 빠른 테스트와 디버깅이 가능합니다.\n- **객체지향 및 함수형 프로그래밍 지원:** 다양한 프로그래밍 패러다임을 지원합니다.\n\n이러한 이유로 파이썬은 입문자부터 전문가까지 널리 사용되는 프로그래밍 언어입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 245, 'prompt_tokens': 17, 'total_tokens': 262, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BlVMdYZ9a76ToLTQrGOrxLiHU4u69', 'service_tier': 'default', 'finish_reason': 'stop', 